In [16]:
import numpy as np
import random
import copy
import math
import operator
import pandas as pd
import time


In [17]:
def distance_between_cities(cities):
    data = dict()
    for index, value in enumerate(cities):
        x1 = cities[index][0]
        y1 = cities[index][1]
        if index + 1 <= len(cities)-1:
            x2 = cities[index+1][0]
            y2 = cities[index+1][1]
            xdiff = x2 - x1
            ydiff = y2 - y1
            dst = (xdiff*xdiff + ydiff*ydiff)** 0.5
            data['Distance from city '+ str(index+1) +' to city ' + str(index+2)] = dst 
        elif index + 1 > len(cities)-1:
            x2 = cities[0][0]
            y2 = cities[0][1]
            xdiff = x2 - x1
            ydiff = y2 - y1
            dst = (xdiff*xdiff + ydiff*ydiff)** 0.5
            data['Distance from city '+ str(index+1) + ' to city ' + str(index +2 -len(cities))] = dst
              
    return data

In [18]:
def total_distance(cities):
    total = sum(distance_between_cities(cities).values())
    return total


In [19]:

def generatePath(cities):
    path = random.sample(cities, len(cities))
    return path


In [20]:
def initialPopulation(cities, populationSize):
    population = [generatePath(cities) for i in range(0, populationSize)]
    return population


In [21]:
def path_fitness(cities):
    total_dis = total_distance(cities)
    fitness = 1 / float(total_dis)
    return fitness


In [22]:
def rankPathes(population):
    fitnessResults = {}
    for i in range(len(population)):
        fitnessResults[i] = path_fitness(population[i])
        
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)



In [23]:
#Roulette wheel Selection Method
def perform_selection(pop, eliteSize):
    df = pd.DataFrame(np.array(pop), columns=["Index","Fitness"])
    df['cumulative_sum'] = df.Fitness.cumsum()
    df['cum_percentage'] = 100*df.cumulative_sum/df.Fitness.sum()
    selected_values = [pop[i][0] for i in range(eliteSize)]
    
    for i in range(len(pop) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(pop)):
            if pick <= df.iat[i,3]:
                selected_values.append(pop[i][0])
                break
                
    return selected_values

In [24]:
#Ordered Crossover
def crossover(mum, dad):
    """Implements ordered crossover"""

    size = len(mum)

    kid1, kid2 = [-1] * size, [-1] * size
    start, end = sorted([random.randrange(size) for _ in range(2)])

    kid1_inherited = []
    kid2_inherited = []
    for i in range(start, end + 1):
        kid1[i] = mum[i]
        kid2[i] = dad[i]
        kid1_inherited.append(mum[i])
        kid2_inherited.append(dad[i])

    current_dad_position, current_mum_position = 0, 0

    fixed_pos = [range(start, end + 1)]       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        test_kid1 = kid1[i]
        if test_kid1==-1: 
            dad_trait = dad[current_dad_position]
            while dad_trait in kid1_inherited:
                current_dad_position += 1
                dad_trait = dad[current_dad_position]
            kid1[i] = dad_trait
            kid1_inherited.append(dad_trait)
        
        test_kid2 = kid2[i]
        if test_kid2==-1: 
            mum_trait = mum[current_mum_position]
            while mum_trait in kid2_inherited:
                current_mum_position += 1
                mum_trait = mum[current_mum_position]
            kid2[i] = mum_trait
            kid2_inherited.append(mum_trait)
       
        i +=1
        

    return kid1, kid2

In [25]:
#Swaping
def mutate(indiv, mutat_rate):
    for exchanged in range(len(indiv)):
        if(random.random() < mutat_rate):
            exchanged_with = int(random.random() * len(indiv))
            
            city1 = indiv[exchanged]
            city2 = indiv[exchanged_with]
            
            indiv[exchanged] = city2
            indiv[exchanged_with] = city1
    return indiv

In [26]:
def mutatation_population(population, mutat_rate):
    mutated_population = [mutate(population[i], mutat_rate) for i in range(len(population))]
    return mutated_population

In [30]:
def main():

  population_size = 150
  number_of_iterations = 1200
  number_of_couples = 20
  mutation_probability = 0.01
  cityList  = [[35.6897,139.6922],[34.693,135.5019],[35.1167,136.9333],[35.4333,139.6333],[33.5903,130.4019],[43.0621,141.3544],[35.0111,135.7669],[34.6913,135.1830],[35.5300,139.7050],[35.8617,139.6453]]
  population = initialPopulation(cityList, population_size)
  best_fitness = 0
  for i in range(number_of_iterations):
    new_population = []
    fit = rankPathes(population)
    temp = fit[0][1]
    temp1 = fit[0][0]
    if temp>best_fitness:
      best_fitness = temp
      min_distance = 1/best_fitness
      best_chromosome = population[temp1]
    
    selected = perform_selection(fit,5)
    for i in range(number_of_couples):
      i = random.randint(0, len(selected)-1)
      j = random.randint(0, len(selected)-1)
      new1, new2 = crossover(population[selected[i]], population[selected[j]])
      new_population = new_population + [new1, new2]
    
    new_population = mutatation_population(new_population, mutation_probability)
    
    for i in range(10):
      new_population = new_population + [population[selected[i]]]
    
    popuation = copy.deepcopy(new_population)
  
  print(best_fitness, min_distance, best_chromosome)
  


    
    

    

In [31]:
start_time = time.time()
main()
print(time.time()-start_time)

0.03088249131484155 32.38080729320625 [[35.4333, 139.6333], [35.6897, 139.6922], [43.0621, 141.3544], [33.5903, 130.4019], [34.6913, 135.183], [34.693, 135.5019], [35.0111, 135.7669], [35.1167, 136.9333], [35.8617, 139.6453], [35.53, 139.705]]
272.94228863716125
